<a href="https://colab.research.google.com/github/fedyasa/CSC/blob/main/prog1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
         
fantasy_zip = zipfile.ZipFile('/content/train.zip')
fantasy_zip.extractall('/content/train')
 
fantasy_zip.close()

In [3]:
import pandas as pd
import numpy as np
import os
import shutil
import random
from tqdm import tqdm

In [4]:
my_dict = {}

df = pd.read_csv('/content/train_tags.csv', header=None) 
my_dict = dict(zip(df[0], df[1]))

In [5]:
op = []
sg = []

for id in my_dict:
  if my_dict[id] == 'open':
    op.append(id)
  if my_dict[id] == 'sunglasses':
    sg.append(id)

In [6]:
os.mkdir('/content/val')
os.mkdir('/content/val/opn')
os.mkdir('/content/val/sun')
os.mkdir('data')
os.mkdir('/content/data/opn')
os.mkdir('/content/data/sun')

In [7]:
path = r"/content/train/train"

for file in os.listdir(path):
  # if random.uniform(0, 6) > 5:
  #   shutil.move(path + '/' + file, '/content/val/' + file)
  if os.path.splitext(file)[0] in op:
    shutil.move(path + '/' + file, '/content/data/opn/' + file)
  if os.path.splitext(file)[0] in sg:
    shutil.move(path + '/' + file, '/content/data/sun/' + file)

In [8]:
for file in tqdm(os.listdir('/content/data/opn')):
  if random.uniform(0, 6) > 5:
    shutil.move('/content/data/opn/' + file, '/content/val/opn/' + file)

for file in tqdm(os.listdir('/content/data/sun')):
  if random.uniform(0, 6) > 5:
    shutil.move('/content/data/sun/' + file, '/content/val/sun/' + file)

100%|██████████| 189/189 [00:00<00:00, 135856.63it/s]


In [9]:
import torchvision
from torchvision import transforms, models
import torch

f = 25

train_transforms_1 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_2 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

train_transforms_3 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_4 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_5 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_6 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

data = '/content/data'
val_dir = '/content/val'

train_dataset_1 = torchvision.datasets.ImageFolder(data, train_transforms_1)
train_dataset_2 = torchvision.datasets.ImageFolder(data, train_transforms_2)
train_dataset_3 = torchvision.datasets.ImageFolder(data, train_transforms_3)
train_dataset_4 = torchvision.datasets.ImageFolder(data, train_transforms_4)
train_dataset_5 = torchvision.datasets.ImageFolder(data, train_transforms_5)
train_dataset_6 = torchvision.datasets.ImageFolder(data, train_transforms_6)

val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

train_dataset = torch.utils.data.ConcatDataset([
                                                train_dataset_1, train_dataset_2,
                                                train_dataset_3, train_dataset_4,
                                                train_dataset_5, train_dataset_6
                                               ])

batch_size = 10

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset)

In [10]:
len(train_dataloader), len(val_dataloader)

(195, 54)

In [11]:
from torch.nn.modules.loss import CrossEntropyLoss
import torch.nn as nn
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.L1 = nn.Linear(30*32, 512)
    self.R1 = nn.ReLU()
    self.L2 = nn.Linear(512, 256)
    self.R2 = nn.ReLU()
    self.L3 = nn.Linear(256, 64)
    self.R3 = nn.ReLU()
    self.L4 = nn.Linear(64, 2)
  def forward(self, input):
    x = self.L1(input)
    x = self.R1(x)
    x = self.L2(x)
    x = self.R2(x)
    x = self.L3(x)
    x = self.R3(x)
    x = self.L4(x)
    return x

model = Model()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0003)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [12]:
def train(model, loss, optimizer, num_epochs = 60):
  for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                model.train()  # Set model to training mode

            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                inputs = inputs.reshape(inputs.shape[0], -1)  # 3*30*32 -> 1*2880

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

  return model

In [13]:
train(model, loss, optimizer)

Epoch 0/59:


100%|██████████| 195/195 [00:02<00:00, 67.69it/s]

train Loss: 0.6845 Acc: 0.5438



100%|██████████| 54/54 [00:00<00:00, 706.37it/s]

val Loss: 0.6308 Acc: 0.6852
Epoch 1/59:


...


100%|██████████| 195/195 [00:05<00:00, 34.53it/s]

train Loss: 0.1617 Acc: 0.9282



100%|██████████| 54/54 [00:00<00:00, 438.81it/s]

val Loss: 0.0438 Acc: 0.9815


Model(
  (L1): Linear(in_features=960, out_features=512, bias=True)
  (R1): ReLU()
  (L2): Linear(in_features=512, out_features=256, bias=True)
  (R2): ReLU()
  (L3): Linear(in_features=256, out_features=64, bias=True)
  (R3): ReLU()
  (L4): Linear(in_features=64, out_features=2, bias=True)
)

In [14]:
fantasy_zip = zipfile.ZipFile('/content/test.zip')
fantasy_zip.extractall('/content')
 
fantasy_zip.close()

In [15]:
from PIL import Image
a = iter(os.listdir("/content/test"))
file = next(a)
image_input = val_transforms(Image.open("/content/test/" + file)).to(device)
image_input.shape
model.eval()
model(image_input.reshape(-1))

tensor([ 0.9748, -0.8202], grad_fn=<AddBackward0>)

In [16]:
test_predictions = []
test_img_paths = []

for file in tqdm(os.listdir("/content/test")):
    inputs = val_transforms(Image.open("/content/test/" + file)).to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs.reshape(-1))
    test_predictions.append(
        torch.nn.functional.softmax(preds).data.cpu().numpy())
    test_img_paths.append(file)

  0%|          | 0/22140 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
100%|██████████| 22140/22140 [00:16<00:00, 1325.01it/s]


In [17]:
print(len(test_img_paths))
print(len(test_predictions))

22140
22140


In [58]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'sunglasses_probability': test_predictions})

In [29]:
submission_df

,id,sunglasses_probability
0,19529.jpg,"[0.85753703, 0.14246298]"
1,7696.jpg,"[0.77621037, 0.22378966]"
2,19554.jpg,"[0.9998043, 0.00019568758]"
3,16112.jpg,"[7.667701e-05, 0.99992335]"
4,13624.jpg,"[0.22123383, 0.7787662]"
...,...,...
22135,21615.jpg,"[0.97735524, 0.022644784]"
22136,10164.jpg,"[2.395501e-07, 0.99999976]"
22137,19661.jpg,"[0.086935356, 0.91306466]"
22138,9709.jpg,"[0.67865825, 0.32134172]"


In [23]:
os.mkdir('/content/result')

In [59]:
for i in range(0, 22140):
  submission_df['sunglasses_probability'][i] = submission_df['sunglasses_probability'][i][1]
#.map(lambda pred: 'open' if pred[0] > 0.5 else 'sunglasses')
submission_df['id'] = submission_df['id'].str.replace('/content/result/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,sunglasses_probability
id,
19529,0.142463
7696,0.22379
19554,0.000196
16112,0.999923
13624,0.778766
5882,0.002543


In [64]:
submission_df.index = submission_df.index.astype(int)
submission_df = submission_df.sort_index()
submission_df.head(n=6)

,sunglasses_probability
id,
0,0.000213
1,0.96601
2,0.251702
3,0.010535
4,0.000818
5,0.16246


In [66]:
submission_df.to_csv('Fedor_Sergeev.csv')